[current dir](.)

In [14]:
# filename = 'inputs/Day17-example.txt'
filename = 'inputs/Day17.txt'
lines = open(filename).read().splitlines()

print(len(lines), lines[:10])

1470 ['x=581, y=396..399', 'y=1491, x=566..573', 'y=1470, x=599..601', 'y=980, x=526..541', 'y=1173, x=487..497', 'y=402, x=616..623', 'y=361, x=534..577', 'x=461, y=1151..1163', 'x=530, y=431..443', 'x=426, y=976..980']


In [2]:
def dropFrom(grid, x, y):
    if y >= len(grid)-1: # out of grid
        return
    if grid[y+1][x] == '.': # drop down
        grid[y+1][x] = '|'
        dropFrom(grid, x, y+1)
    if grid[y+1][x] in '#~' and grid[y][x+1] == '.': # go right
        grid[y][x+1] = '|'
        dropFrom(grid, x+1, y)
    if grid[y+1][x] in '#~' and grid[y][x-1] == '.': # go left
        grid[y][x-1] = '|'
        dropFrom(grid, x-1, y)
    if iswalled(grid, x, y):# if walled, fill to nearby walls
        filltowalls(grid, x, y)

In [3]:
def iswalled(grid, x, y): # walled if we reach # before an empty tile or end of grid, on both sides
    for ix in range(x, -1, -1):
        if grid[y][ix] == '#':
            break
        elif grid[y+1][ix] == '.' or ix <= 0:
            return False
    for ix in range(x, len(grid[0])-1):
        if grid[y][ix] == '#':
            break
        elif grid[y+1][ix] == '.' or ix >= len(grid[0])-1:
            return False
    return True

In [4]:
def filltowalls(grid, x, y): # expand with ~ until both walls
    for ix in range(x, 0, -1):
        if grid[y][ix] == '#':
            break
        grid[y][ix] = '~'
    for ix in range(x, len(grid[0])-1):
        if grid[y][ix] == '#':
            break
        grid[y][ix] = '~'

In [5]:
import re

def parseclay():
    claylines = []
    for line in lines:
        mx, my = re.search('x=(?P<xS>\d+)(\.\.(?P<xE>\d+))?', line),  re.search('y=(?P<yS>\d+)(\.\.(?P<yE>\d+))?', line)
        claylines.append({
            'x': {'start': int(mx.group('xS')), 'end': int(mx.group('xE')) if mx.group('xE') != None else int(mx.group('xS'))},
            'y': {'start': int(my.group('yS')), 'end': int(my.group('yE')) if my.group('yE') != None else int(my.group('yS'))}
        })
    return claylines

In [6]:
def zerobase(claylines):
    minX = min([clayline['x']['start'] for clayline in claylines])
    for i in range(len(claylines)):
        for k in claylines[i]['x']:
            claylines[i]['x'][k] -= minX-1
    return minX, claylines

In [7]:
def setupgrid(xDim, yDim, claylines, pumpX):
    grid = [['.' for _ in range(xDim+1)] for _ in range(yDim+1)]
    for p in claylines:
        for x in range(p['x']['start'], p['x']['end']+1):
            for y in range(p['y']['start'], p['y']['end']+1):
                grid[y][x] = '#'
    grid[0][pumpX] = '+'
    return grid

In [8]:
def simulate():
    claylines = parseclay() # parse input to areas of clay
    minX, claylines = zerobase(claylines) # make coordinates zero-based

    pumpX = 500-minX #calculate new zero-based X position for the pump
    org = [[clayline[axis][ext] for clayline in claylines] for axis in ['x', 'y'] for ext in ['start', 'end']]
    (minX, maxX, minY, maxY) = (min(org[0]), max(org[1]), min(org[2]), max(org[3])) # assign mix/max X/Y
    grid = setupgrid(maxX+1, maxY, claylines, pumpX) # setup the grid
    
    dropFrom(grid, pumpX, 0) # simulate
    return grid[minY:]

Part 1

In [15]:
grid = simulate()
print(sum([int(val in '~|') for l in grid for val in l]))

30635


Part 2

In [16]:
grid = simulate()
print(sum([int(val == '~') for l in grid for val in l]))

25094
